In [ ]:
import numpy as np 
import json
from underthesea import sent_tokenize
json_path = '/home/thangquang/CODE/CTAI_MachineLearning/data/ise-dsc01-train_new.json'


with open(json_path, 'r', encoding='utf-8') as f:
    data = json.loads(f.read())

keys = list(data.keys())

In [40]:
sample_data = data[keys[0]]
sample_data

{'context': 'Phát biểu tại lễ ký kết vào ngày 17/11, Giám đốc điều hành kiêm đồng sáng lập FABA Technology - Nguyễn Thành Hưng nhận định trong bối cảnh "bình thường mới" như hiện tại, mô hình đào tạo 100% online của FUNiX là một trong những cách có thể giải bài toán khan hiếm nhân lực của ngành công nghệ thông tin.\n\nÔng cũng chia sẻ việc trở thành đối tác đào tạo và tuyển dụng cùng FUNiX sẽ giúp FABA Technology đạt mục tiêu trở thành công ty offshore development chất lượng nhất trong khu vực.\n\nQua hợp tác này, FABA kỳ vọng sẽ có ứng viên chất lượng cho rất nhiều vị trí công việc đang tìm kiếm như Automation Tester, UI/UX Designer, Web Developer, Mobile Developer ở nhiều cấp độ... "Chúng tôi hoàn toàn tin tưởng vào chất lượng đào tạo của FUNiX" - ông nhấn mạnh.\n\nĐể thu hút và giữ chân nhân tài trong một thị trường nhân sự IT khốc liệt, FABA không giới hạn mức lương, luôn cập nhật để dải lương đạt mức cạnh tranh tốt so với thị trường. Công ty cũng có chế độ đãi ngộ tốt, có lương th

In [74]:
sample_context = sample_data['context']
sample_verdict = sample_data['verdict']
sample_evidence = sample_data['evidence']
sample_claim = sample_data['claim']

In [16]:
len(data)

37967

In [92]:
# lặp qua tất cả các sample, kiểm tra xem mỗi trường có bao nhiêu thông tin bị missing
missing_values = {
    'context': 0,
    'claim': 0,
    'evidence': 0,
    'verdict': 0
}
missing_idx = []

for idx, sample in enumerate(data.values()):
    for key in missing_values.keys():
        if not sample[key]:
            missing_values[key] += 1
            missing_idx.append(idx)


In [91]:
missing_values

{'context': 0, 'claim': 0, 'evidence': 12583, 'verdict': 0}

In [96]:
data[keys[missing_idx[0]]]

{'context': 'Thay mặt 10 thanh niên tiêu biểu nhận bằng khen của Bộ trưởng Quốc phòng, thượng úy Lê Hảo (Trợ lý Thanh niên, Phòng Chính trị, Bộ Tư lệnh TP HCM) cho biết sau khi tốt nghiệp trường Sĩ quan Chính trị năm 2020, anh được phân công về Sư đoàn Gia Định. Đây là thời điểm TP HCM bước vào giai đoạn chống dịch Covid-19 cam go nhất.\n\nĐược giao tham gia tổ công tác đặc biệt làm nhiệm vụ tiếp nhận xử lý thi hài của nạn nhân không may qua đời do mắc Covid-19, anh xác định đây nhiệm vụ khó khăn nhưng vô cùng thiêng liêng. Dù nguy cơ lây nhiễm cao, thượng úy Hảo cùng đồng đội động viên nhau ngày đêm làm việc, hoàn thành nhiệm vụ được giao.\n\n"Tôi hứa tuyệt đối trung thành, sẵn sàng chiến đấu, hy sinh bảo vệ Tổ quốc, Đảng, Nhà nước, nhân dân và chế độ Xã hội chủ nghĩa", thượng úy Hảo nói.\n\nĐây là đài rađa có các tính năng tương đương và nhiều điểm vượt trội so với các sản phẩm rađa nước ngoài. Đơn cử như khả năng cơ động - yếu tố sống còn trong tác chiến hiện đại; kích thước ăngten 

In [93]:
missing_idx[0]

12786

In [14]:
import re
import unicodedata
from pyvi import ViTokenizer

def preprocessing(text: str) -> str:
    # chuẩn hóa unicode
    text = unicodedata.normalize('NFC', text)
    # đưa về chữ thường
    text = text.lower()
    # xoa cac html tag
    text = re.sub(r'<[^>]+>', '', text)
    # xoa khoang trang
    text = re.sub(r'\s+', ' ', text).strip()
    # MINIMAL cleaning - chỉ xóa emoji
    # Giữ lại punctuation vì có thể quan trọng cho meaning
    text = re.sub(r'[^\u00C0-\u1EF9a-zA-Z0-9\s.,!?;:()\-"\'%]', '', text)
    # tokenize theo tieng viet
    text = ViTokenizer.tokenize(text)
    return text

In [15]:
preprocessing(data[keys[0]]['claim'])

'ngoài việc không giới_hạn_mức lương , công_ty còn có đãi_ngộ tốt , có phụ_cấp , lương tháng 13,14 , ...'

In [97]:
def prepare_data(sample_data):
    context = sent_tokenize(preprocessing(sample_data['context']))
    claim = preprocessing(sample_data['claim'])
    evidence = preprocessing(sample_data['evidence']) if sample_data['evidence'] else ''
    verdict = sample_data['verdict']

    return {
        'context': context,
        'claim': claim,
        'evidence': evidence,
        'verdict': verdict
    }

In [ ]:
new_data = [prepare_data(sample_data) for sample_data in data.values()]

In [100]:

## lưu new_data ra ổ đĩa
json.dump(new_data, open('/home/thangquang/CODE/CTAI_MachineLearning/data/ise-dsc01-train_new_preprocessed.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [99]:
new_data[9]

{'context': ['pgs .',
  'ts nguyễn quang liệu , hiệu_trưởng trường thpt chuyên_khoa_học xã_hội và nhân_văn , hôm 155 cho biết có 1.579 thí_sinh đăng_ký vào ba lớp chuyên của trường , trong khi chỉ_tiêu là 136 .',
  'lớp chuyên ngữ_văn có 970 học_sinh đăng_ký - cao nhất từ trước đến nay .',
  'với chỉ_tiêu 66 , lớp này có tỷ_lệ chọi 114,7 , tăng so với mức 19 của năm_ngoái .',
  'như_vậy , tính trung_bình thì cứ 15 thí_sinh mới có một em trúng_tuyển .',
  'hai lớp chuyên lịch_sử và địa_lý cùng tuyển 35 học_sinh , lần_lượt nhận 317 và 292 hồ_sơ , tỷ_lệ chọi là 19,05 và 18,3 , cũng tăng so với năm 2022 .',
  'ngoài ba lớp chuyên , trường tuyển 34 học_sinh hệ chất_lượng cao .',
  'tất_cả thí_sinh phải làm bốn bài thi_viết hôm 46 , gồm ngữ_văn , toán , tiếng anh và môn chuyên .',
  'trong đó , đề môn ngữ_văn và môn chuyên theo hình_thức tự luận , thời_gian làm bài lần_lượt 90 và 150 phút .',
  'thí_sinh thi trắc_nghiệm môn toán trong 50 phút và tiếng anh 45 phút .',
  'lệ_phí đăng_ký và thi

## tạo data cho triplet loss

In [106]:
import hashlib, json, random, tqdm, os
random.seed(42)

def hash_context(sent_list):
    """Tạo ID duy nhất cho một context (list sentence)."""
    txt = " ".join(sent_list)
    return hashlib.md5(txt.encode("utf-8")).hexdigest()


context_bank = {}           # context_id  -> list(sentence)
examples_by_context = []   # lưu meta cho từng claim

for item in new_data:
    cid = hash_context(item['context'])
    if cid not in context_bank:
        context_bank[cid] = item['context']
    examples_by_context.append({
        'cid': cid,
        'claim': item['claim'],
        'evidence': item['evidence'],
        'verdict': item['verdict']
    })

In [107]:
def sample_negatives(curr_cid, ev_sentence, k_in=1, k_out=1):
    negs = []
    
    # a) Cùng context nhưng ≠ evidence
    in_context = [s for s in context_bank[curr_cid] if s != ev_sentence]
    if in_context:
        negs.extend(random.sample(in_context, min(k_in, len(in_context))))
    
    # b) Khác context
    while len(negs) < k_in + k_out:
        cid = random.choice(list(context_bank.keys()))
        if cid == curr_cid: continue
        negs.append(random.choice(context_bank[cid]))
    return negs

triplets = []
for ex in examples_by_context:
    if ex['verdict'] == 'NEI' or ex['evidence'] == '':
        continue
    negs = sample_negatives(ex['cid'], ex['evidence'], k_in=1, k_out=1)
    for neg in negs:          # tạo 1 triplet cho mỗi negative
        triplets.append({
            "anchor":   ex['claim'],
            "positive": ex['evidence'],
            "negative": neg
        })

# Lưu
with open("train_triplets.jsonl", "w", encoding="utf8") as f:
    for row in triplets:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")


In [108]:
triplets[0]

{'anchor': 'ngoài việc không giới_hạn_mức lương , công_ty còn có đãi_ngộ tốt , có phụ_cấp , lương tháng 13,14 , ...',
 'positive': 'công_ty cũng có chế_độ đãi_ngộ tốt , có lương tháng 13 , 14 , phụ_cấp . .',
 'negative': 'ông cũng chia_sẻ việc trở_thành đối_tác đào_tạo và tuyển_dụng cùng funix sẽ giúp faba technology đạt mục_tiêu trở_thành_công_ty offshore development chất_lượng nhất trong khu_vực .'}